In [2]:
"""
THIS CODE MAPS FRAME DATA TO TRUE ENGLISH TRANSLATION

It maps all the sentences to corresponing frame data:
For example: 
    'Hi I am from new york' is mapped to data between frames 20 and 234.


The frame data used in this file is gathered present in:
    key_points_5.npy (If using 5 features) --> gathered using toJSON_PCA_5feat.ipynb
    key_points_42.npy (If using 42 features) --> gathered using toJSON_PCA_m.ipynb
"""
import numpy as np
import pandas as pd

In [21]:
# words to ignore from true englush translation
remove = ['am','is','are', 'was', 'were','a','an','the','','during','usually','on','to','at','it\'s','it','and','that','so','made',\
          'them','about','of','so','by','will','would','shall','should']
li = [remove[0:14],remove[14:]]
pd.DataFrame(li)
# pd.DataFrame(remove[14:]).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,am,is,are,was,were,a,an,the,,during,usually,on,to,at,None
1,it's,it,and,that,so,made,them,about,of,so,by,will,would,shall,should


In [22]:
# IGNORE THIS

# signer_names=['SIA02','SIB01','SIC02']
# remove = ['am','is','are', 'was', 'were','a','an','the','','during','usually','on','to','at','it\'s','it','and','that','so','made',\
#           'them','about','of','so','by','will','would','shall','should']
# story = 'story'
# for i in range(1,20):
#     if i<10:
#         s_no = '0'+str(i)
#     else:
#         s_no = str(i)
#     s_no = story+s_no
#     try:
#         trans_dir = 'Translation/'+signer_names[0]+'-'+s_no+'-translation.txt'
#         gloss_dir = 'Gloss/'+signer_names[0]+'-'+s_no+'-gloss.txt'
#         text = open('Translation/'+signer_names[0]+'-'+s_no+'-translation.txt')
#         text = text.read()
#         text = text.split('.')
#         print('\n'+s_no+'\nSentences:\n')
#         for i in text:
#             print(i)
#         print("\nActual Count: ",get_end_word_count(text)[1])
#         print("Word Count: ",get_end_word_count(text)[0])
#         g_txt = open('Gloss/'+signer_names[0]+'-'+s_no+'-gloss.txt')
#         g_txt = g_txt.read().split('\n')
#         print("Gloss length: ",len(g_txt))
#         print("words ", get_end_word_count(text)[2])
#         indexes = get_index(text)
#         print("Index: ", indexes)
#         match_gloss_sentence(gloss_dir,text,indexes)
#     except:
#         pass
    
    
# #     gloss_txt = np.genfromtxt('Gloss/'+signer_names[0]+'-'+s_no+'-gloss.txt')
# #     g_txt = open('Gloss/'+signer_names[0]+'-'+s_no+'-gloss.txt')
# #     g_txt = g_txt.read().split('\n')
# #     print("Gloss length: ",len(g_txt))
# #     print("Gloss Count: ",gloss_txt)

In [16]:
def match_sen_frames(sen_data, frame_data):
    sen_frame_data = []
    for i in sen_data:
        start = int(i[0])
        end = int(i[1])
        
        if start < frame_data.shape[0] and end> frame_data.shape[0]:
            end = frame_data.shape[0]-1  
        if start>=frame_data.shape[0]:
            pass
        else:
            sen_fr = frame_data[start:end]
        sen_frame_data.append([sen_fr,i[2]])
        
    return np.array(sen_frame_data)

In [17]:
def get_end_word_count(text):
    count = 0
    act_ct = 0
    words = []
    for sentence in text:
        for word in sentence.lower().split(' '):
            if word not in remove:
                words.append(word)
                count+=1
            act_ct+=1
    return count, act_ct, words

def get_index(text):
    count = 0
    words = []
    for sentence in text:
#         print('Sentence: ', sentence,'--',len(sentence))
        if len(sentence)!=0 and sentence.strip()!='':
            count = count + get_end_word_count([sentence])[0]
            words.append(count)
            
#         print('words: ',words)
    return words

In [18]:
import re
def preprocess(temp):
    li = []
    for i in temp:
        if re.search('[a-zA-Z]', i) != None:
            li.append(i)
    return li

In [23]:
def read_gloss(direc):
    g_txt = open(direc)
    g_txt = g_txt.read().split('\n')
    gloss = []
    for i in g_txt:
        x = i.split('\t')
        if x[0] != '':
            gloss.append(x)
    return gloss


# To change the window length for the mapping, change the 'window' attribute in the below function.
# Different window lengths can be used as a method of data augmentation
def match_gloss_sentence(gloss_dir, text, indexes):
    gls = read_gloss(gloss_dir)
    sen_start_end = []
    window = 2
    
    start_pt = 0
    end_pt = indexes[0]
    
    st_id_track = 0
    
    while end_pt <= len(gls)-1:
        start_pt_w = start_pt
        end_pt_w = end_pt
        
        
        if start_pt!=0 and start_pt-window >=0:
            start_pt_w = start_pt - window
            
        if end_pt+window <= len(gls)-1:
            end_pt_w = end_pt + window
        
        if end_pt == len(gls)-1:
            end_pt_w = end_pt +1
            
        if st_id_track == len(indexes)-1 and end_pt_w < len(gls)-1:
            end_pt = len(gls)-1
            end_pt_w = end_pt
        

        
#         print('\nStart Point: ',start_pt_w)
#         print('End Point: ', end_pt_w)
        try:
            count = 0
            sen_start = 0
            sen_end = 0
            for i in range(start_pt_w,end_pt_w):
                if i == start_pt_w: 
                    sen_start = gls[i][1]
                if i == end_pt_w-1:
                    sen_end = gls[i][2]
#                 print(gls[i])
            
#             print('Frame start: ', sen_start)
#             print('Frame end: ', sen_end)
            sen_start_end.append([sen_start, sen_end,text[st_id_track]])
        except:
            pass
            
#         print(text[st_id_track])
        st_id_track +=1
        start_pt = end_pt
        if st_id_track < len(indexes):
            end_pt = indexes[st_id_track]
            if end_pt > len(gls)-1:
                end_pt = len(gls)-1
        else:
            end_pt = len(gls)+1
#         print('Update end: ', end_pt)
    
    return sen_start_end
        
        
    
    
# direc = 'Gloss/'+signer_names[0]+'-'+s_no+'-gloss.txt'
# read_gloss(direc)
# match_gloss_sentence(direc,text,indexes)

# ind = [4, 8, 14, 19, 25, 36, 38, 42, 49, 61, 66, 73]



In [14]:
loaded_data = np.load('key_points_42.npy')
sentence_frame_mapped_data = []
count = 0
for key in loaded_data:
    for k in key:
        try:
            f_n = k.strip('front_x264')
            trans_dir = 'Translation/'+f_n+'translation.txt'
            gloss_dir = 'Gloss/'+f_n+'gloss.txt'
#             print(trans_dir)
            text = open(trans_dir)
            text = text.read()
            text = text.split('.')
            text = preprocess(text)
            print('\n'+f_n+'\nSentences:\n')
#             for i in text:
#                 print(i)
#             print("\nActual Count: ",get_end_word_count(text)[1])
#             print("Word Count: ",get_end_word_count(text)[0])
            g_txt = open(gloss_dir)
            g_txt = g_txt.read().split('\n')
#             print("Gloss length: ",len(g_txt))
#             print("words ", get_end_word_count(text)[2])
            indexes = get_index(text)
#             print("Index: ", indexes)
            sen_indexes = match_gloss_sentence(gloss_dir,text,indexes)
#             print(sen_indexes)
            res = match_sen_frames(sen_indexes, key[k])
            if count == 0:
                sentence_frame_mapped_data = res
            else:
                sentence_frame_mapped_data = np.vstack((sentence_frame_mapped_data,res))
            print(sentence_frame_mapped_data.shape)
            count+=1
        except Exception as e:
            print(e)
            pass


SIA02-story01-
Sentences:

(12, 2)

SIB01-story01-
Sentences:

(22, 2)

SIC02-story01-
Sentences:

(95, 2)

SIA02-story02-
Sentences:

(101, 2)

SIB01-story02-
Sentences:

(109, 2)

SIC02-story02-
Sentences:

(127, 2)

SIA02-story05-
Sentences:

(132, 2)

SIB01-story05-
Sentences:

(138, 2)

SIC02-story05-
Sentences:

(150, 2)

SIA02-story06-
Sentences:

(154, 2)

SIB01-story06-
Sentences:

(169, 2)

SIC02-story06-
Sentences:

(194, 2)

SIA02-story09-
Sentences:

(204, 2)

SIB01-story09-
Sentences:

(212, 2)

SIC02-story09-
Sentences:

(225, 2)

SIA02-story12-
Sentences:

(235, 2)

SIB01-story12-
Sentences:

(251, 2)

SIC02-story12-
Sentences:

(275, 2)

SIA02-story13-
Sentences:

(282, 2)

SIB01-story13-
Sentences:

(296, 2)

SIC02-story13-
Sentences:

(304, 2)

SIA02-story15-
Sentences:

(314, 2)

SIB01-story15-
Sentences:

(320, 2)

SIC02-story15-
Sentences:

(347, 2)

SIA02-story17-
Sentences:

(355, 2)

SIB01-story17-
Sentences:

(367, 2)

SIC02-story17-
Sentences:

(380, 2)

SIA

In [17]:
# here, w2 in the filename means, window length is 2
np.save('sentence_frame_mapped_w2_42.npy', sentence_frame_mapped_data)

In [26]:
# all the code below this cell is just for testing and can be ignored

In [24]:
print(text)

['A prisoner managed to escape from jail and went across Hungary', " This person wasn't sure where to go", ' So, the prisoner asked a man who happened to be a cop, "How can I get through here?" The cop was thrilled to see the prisoner and pretended to help the prisoner to find a way out', ' The cop lead the prisoner to the police station and arrested the prisoner', ' What a fool he was', '', '', ' actually, a big fool!\n\n\n\n']


In [25]:
temp = 'My background? Ok, my name is J... Excuse me. My name is Jeremy. I was raised here in New York and attended PS 47, a school for the Deaf. All of my family members are hearing, except my brother who is also Deaf. I have a hearing sister. I am the youngest and third child. I attended Gallaudet for a year and then withdrew. I worked at an Apple store, and then worked at the Whitney Museum. I edit films there. '
temp = temp.split('.')
print(temp)
for i in temp:
    print(i)

['My background? Ok, my name is J', '', '', ' Excuse me', ' My name is Jeremy', ' I was raised here in New York and attended PS 47, a school for the Deaf', ' All of my family members are hearing, except my brother who is also Deaf', ' I have a hearing sister', ' I am the youngest and third child', ' I attended Gallaudet for a year and then withdrew', ' I worked at an Apple store, and then worked at the Whitney Museum', ' I edit films there', ' ']
My background? Ok, my name is J


 Excuse me
 My name is Jeremy
 I was raised here in New York and attended PS 47, a school for the Deaf
 All of my family members are hearing, except my brother who is also Deaf
 I have a hearing sister
 I am the youngest and third child
 I attended Gallaudet for a year and then withdrew
 I worked at an Apple store, and then worked at the Whitney Museum
 I edit films there
 


In [74]:
da = np.load('sentence_frame_mapped.npy')

for i in range(1,2):
    print(da[i])

[array([[-519.00908718,  -91.46209743,  -39.70546752, -172.41683612,
        -235.2344303 ],
       [-239.44054254, -118.33488168, -114.18714284, -238.1086968 ,
        -232.55476311],
       [-546.86466186, -132.94569837,  -77.53829564, -146.34550112,
        -223.42111473],
       [-550.67763673, -121.37938835,  -90.84958127, -138.50820058,
        -216.72664371],
       [-195.25717607, -128.7176959 , -112.24071994, -243.0779694 ,
        -231.62127647],
       [-310.59222313, -155.46991676, -160.94911607, -227.80565191,
        -222.04633665],
       [-578.91273914, -115.33520596,  -92.8099466 , -122.73165423,
        -212.80292159],
       [-573.09553064, -130.25841382,  -83.20990674, -114.03505954,
        -215.04111757],
       [-572.47915315, -137.28915655,  -68.72257302, -113.72685848,
        -216.52187106],
       [-610.4358942 , -126.32545752,  -77.11021788,  -97.20687615,
        -214.34106096],
       [-610.11179185, -123.79087537,  -65.1104137 ,  -94.01108106,
        -21